In [ ]:
ivalis_artif

In [ ]:
ivalis_artif_tb


# traceback catch locals

In [ ]:
q = 11
def test(arg):
    q = 33
    s = 0/0.
    return s

import sys, traceback

try:
    test(100)
except:
    exc_info = sys.exc_info()
    exc = traceback.TracebackException(*exc_info, capture_locals=True)
    
    # exc_info == (ZeroDivisionError, 
    # ZeroDivisionError('float division by zero'),
    # <traceback at 0x7f0183171e00>)
    
    local_vars_in_fun = exc_info[2].tb_next.tb_frame.f_locals
    display(local_vars_in_fun)
    exc_info[2].tb_lineno # line number, I can also get line number inside function using tb_next.tb_lineno
    exc_info[2].tb_lasti # idk what this number means

In [ ]:
exc.__dict__

In [ ]:
exc_info

In [ ]:
exc.stack  # gives strings which identify where in stack the exc happened (inc when in jupyter)

In [ ]:
exc_info[2].tb_next.tb_lineno

In [ ]:
exc.stack[0]

# Debug resave

In [ ]:
%load_ext autoreload
%autoreload 2

#%debug
%run ../run/resave.py -r S01_off_hold --read_type 'hires-raw' --to_perform notch

In [ ]:
upre.readInfo(fname_noext, raw, sis=[1,2],
             check_info_diff = 1, bandpass_info=0 )


In [ ]:
#raw.get_channel_types()

In [ ]:
raw.info['chs']

In [ ]:
    if recalc_artif:
        r = upre.recalcMEGArtifacts(fname_noext,
            raw, thr_mult=MEG_artif_thr_mult, use_mean=MEG_thr_use_mean,
            flt_thr_mult=MEG_flt_artif_thr_mult,
            flt_use_mean=MEG_flt_thr_use_mean,
            lowest_freq_to_keep=lowest_freq_to_keep,
            n_jobs=n_jobs, raw_flt = None, raw_notched = raw_notched,
            savedir=data_dir_input, notch_freqsToKill=freqsToKill )
        anns_MEG_artif, anns_MEG_artif_flt, anns_icaMEG_artif, anns_MEG_artif_muscle  = r

    raw = raw_notched



In [ ]:
raw.info['chns']

# Debug artifacts and filtering

In [ ]:
sfreq_hires = 2000
#sfreq = 256
sfreq = 250
duration = 10
artif = [3,4]
noise_size = 1e-10

times = np.arange(0,duration,1/sfreq_hires)
dat = np.random.uniform(low=-1,high=1,size=(1,len(times))) * noise_size
dat[0,artif[0]*sfreq_hires:artif[1]*sfreq_hires] += 20

plt.plot(times,dat[0])

In [ ]:
new_info = mne.create_info(
    ch_names=['EEG1'],
    ch_types=['eeg'],
    sfreq=sfreq_hires)

raw = mne.io.RawArray(dat/1e6, new_info)

In [ ]:
%matplotlib qt

In [ ]:
raw.plot();

In [ ]:
raw_resampled = raw.copy()

In [ ]:
raw_resampled.resample(sfreq)

In [ ]:
raw_resampled.plot();

In [ ]:
help(raw_resampled.resample)

# Debug artifact & movement collection (to be used for source reconstruction)

In [ ]:
duration = 40
sfreq = 256


                    
bins = fillBinsFromAnns([anns, anns2], duration, sfreq)

In [ ]:
cvlskip,pairs = utils.getIntervals(bininds=None,width=0,percentthr=1,inc=1,minlen=2,cvl=bins,
                   min_dist_between=1,include_short_spikes=1)

In [ ]:
for s,e in pairs:
    ss,ee = s/sfreq,e/sfreq
    print(ss,ee)

In [ ]:
ann_merged= mergeAnns([anns, anns2], duration, sfreq)

In [ ]:
ann_merged.onset, ann_merged.duration

In [ ]:
%run -i run_collect_artifacts.py -r S01_off_hold


In [ ]:
ann_list

In [ ]:
merged_anns

# Hjorth

In [ ]:
import numpy as np
import utils

In [ ]:
z = np.arange(24)
#z   #array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
s = utils.stride(z, (10,), (5,))

np.mean(s, axis=1), np.std(s, axis=1)  # array([ 3.,  5.,  7.,  9., 11., 13., 15.])

In [ ]:
ss.shape

In [ ]:
zz = np.vstack( [np.arange(24), np.arange(24)] )
#z   #array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
ss = stride(zz, (1,3,), (1,2,))

np.sum(ss, axis=-1), np.var(ss, axis=-1, ddof=1)  # array([ 3.,  5.,  7.,  9., 11., 13., 15.])

In [ ]:
import pandas as pd
pd.Series(zz[0]).rolling(3).var()

In [ ]:

#I failed to point out that you can create an output that you could append as a column into pandas. Going back to the original definitions used above

nans = np.full_like(z, np.nan, dtype='float')  # z is the 20 number sequence
means = np.mean(s, axis=1)   # results from the strided mean
# assign the means to the output array skipping the first and last 3 and striding by 2

nans[3:-3:2] = means        

nans # array([nan, nan, nan,  3., nan,  5., nan,  7., nan,  9., nan, 11., nan, 13., nan, 15., nan, nan, nan, nan])

In [ ]:
windowsz = 256
nbins = windowsz * 7
skip = 16
dt = 1/256

windowsz = 3
nbins = windowsz * 3 + 3
skip = 2
dt = 1

#dat = np.vstack( [np.arange(nbins), np.arange(nbins)] )
dat = np.random.uniform( size = (2,nbins) )

In [ ]:
import utils
#%debug
act,mob,compl    = utils.Hjorth(dat,dt, windowsz, skip, stride_ver= True)
act2,mob2,compl2 = utils.Hjorth(dat,dt, windowsz, skip, stride_ver= False)
print( act.shape, mob.shape)

In [ ]:
act-act2

In [ ]:
mob-mob2

In [ ]:
compl-compl2

In [ ]:
dat.shape

In [ ]:
window = (1,256); step = (1,32)

In [ ]:
np.aran

In [ ]:
dat.shape[-1] // skip

In [ ]:
(dat.shape[-1] - windowsz) // skip + 1

In [ ]:
(ext.shape[-1] - windowsz) // skip + 1

In [ ]:
tuple( ( np.array(dat.shape) - np.array(window) ) // np.array(step) + 1) + window

In [ ]:
ext = np.pad(dat,[(0,0),(windowsz - skip,0)],mode='edge')
ext.shape

In [ ]:
act2.shape, mob2.shape

In [ ]:
256 * 10    / 256 * (256/32)

In [ ]:
#del utils
import importlib as il
il.reload(utils)
import utils

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
sfreq = 256
sfreq = 512
#sfreq = 1024

#wsz = int( 1 * sfreq )
freqs,n_cycles,Ws,wsz = utils.prepFreqs(min_freq = 3, max_freq = 31, frmults=[1,1,1],
                                       sfreq=sfreq)
skip=32
durs = np.array([1.5, 5, 7])
#durs += 14/sfreq
dats_test = len(durs) * [0]
for duri,dur in enumerate(durs):
    durbin = int(dur*sfreq)
    #dats_test[duri] = np.vstack( [np.arange(durbin), np.arange(durbin)] )
    cur = np.vstack( [np.ones(durbin), np.ones(durbin)] )
    cur[:,::2] = 0
    dats_test[duri] = cur * 1000
    

tfrres_pri = len(durs)*[0]
h_pri= len(durs) * [0]
wsz

In [ ]:
wsz

In [ ]:
#%debug
for dati,dat_test in enumerate(dats_test):
    print(dati,durs[dati], durs[dati]*sfreq, (durs[dati]*sfreq)//skip)
    tfrres_pri[dati] = utils.tfr(dat_test,sfreq,freqs,n_cycles,wsz,decim=skip)
    
    h_pri[dati] = utils.Hjorth(dat_test,1/sfreq,wsz, skip, remove_invalid=True)

In [ ]:
for dati,dat_test in enumerate(dats_test):
    print(dati,durs[dati],durs[dati]*sfreq,
          h_pri[dati][0].shape[-1], h_pri[dati][1].shape[-1],
          tfrres_pri[dati][0].shape[-1],tfrres_pri[dati][1].shape[-1])

In [ ]:
dati = 0
wbd_H = h_pri[dati][-1]
wbd_tfr = tfrres_pri[dati][1]

In [ ]:
wbd_H[0], wbd_H[1]

In [ ]:
wbd_H

In [ ]:
wbd_tfr[0], wbd_tfr[1]

In [ ]:
windowsz/sfreq

In [ ]:
dats_test[0].shape

In [ ]:
768 // skip

In [ ]:
np.max( [ np.max( [len(w) for w in ww] ) for ww in Ws ] )


In [ ]:
wsz//skip

In [ ]:
ndb = 8192
wsz = 256
skip =32
padlen = wsz - skip
pred = np.arange(ndb + padlen)
wnds = utils.stride(pred, (wsz,), (skip,) )
dd = np.vstack([pred,pred])
#wnds = utils.stride(dd, (1,wsz), (1,skip) )
wbds = np.vstack([ wnds[:,0],wnds[:,-1] ] )

a,m,c,wbds_H = utils.Hjorth(dd,1/sfreq,wsz, skip, remove_invalid=True)

print( wnds.shape,  wbds.shape, wsz//skip )

In [ ]:
wbds_H

In [ ]:
wbds

In [ ]:
wbds[0,16:].shape

In [ ]:
wbds

In [ ]:
windowsz

In [ ]:
np.arange(0, 768 - wsz, skip)

In [ ]:
tfr,wbd = tfrres_pri[0]
act,mob,compl,wbdh = h_pri[0]

In [ ]:
wbd, tfr.shape

In [ ]:
wbdh, act.shape

In [ ]:
wbd.shape

In [ ]:
tfr.shape

In [ ]:
tfrres_pri[0].shape

In [ ]:

n_times = tfrres_pri[0].shape[-1]
wsz = sfreq
startind = (n_times + wsz - 1 -  n_times) // 2 # == (wsz - 1) // 2
endind = startind + n_times
#and then we decimate
startind,endind

In [ ]:
durs[0]*sfreq / skip

In [ ]:
m = 2
freqs,n_cycles = utils.prepFreqs(frmults=[m,m,m])

sfreq = 256
Ws = mne.time_frequency.tfr._make_dpss(sfreq, freqs, n_cycles=n_cycles,
                        time_bandwidth=4, zero_mean=False)
lens = []
for ww in Ws:
    for w in ww:
        lens += [len(w)]
ulens = np.unique(lens)
ulensl = len( ulens )
assert ulensl == 1, ulens
print(ulens[0])

In [ ]:
Wmor = mne.time_frequency.tfr.morlet(sfreq, freqs, n_cycles=n_cycles, zero_mean=False)
Ws_mor = [Wmor]

In [ ]:
#list (ntapers) of lists (nfreqs)

In [ ]:
sfreq * 1.5

In [ ]:
freqs

In [ ]:
for ww in Ws:
    print('---')
    for w in ww:
        print(len(w))

In [ ]:
for ww in Ws_mor:
    print('---')
    for w in ww:
        print(len(w))

In [ ]:
len(Ws[0]), len(Ws[0][0])

In [ ]:
wsz = 5
second = wsz
arr = np.ones(10)
res = np.convolve(arr,np.ones(second),mode='same')
res_all = np.convolve(arr,np.ones(second),mode='full')
print(res_all.shape)
display(res_all)

#n_full_window_fits = len(arr) - wsz + 1
#n_right = wsz - 1
#n_left  = wsz - 1
#actual_size = len(arr)
edgbins_start = second-1-1 # second -1 is because we count from zero
edgbins_end = second - edgbins_start 
print(edgbins_start, edgbins_end)
display(res, res[edgbins_start:-edgbins_end])

In [ ]:
%matplotlib inline

In [ ]:
help(mne.time_frequency.tfr_array_multitaper)

In [ ]:
mrks = ['o', '+', '3']
# 6 and 4 are right values for 256
nedgeBins_start = int(durs[0]*sfreq / skip) // 2
nedgeBins_end = int(durs[0]*sfreq / skip) // 2 -1 #- 2

# nedgeBins_start += 1
# nedgeBins_end   += 1

freqi = 4

for shft in range(nedgeBins_end):

    print(nedgeBins_start, nedgeBins_end)
    for duri,dur in enumerate(durs):
        dd = tfrres_pri[duri][0,freqi,nedgeBins_start:-nedgeBins_end]
        #print(dd.shape)
        plt.plot(np.abs( dd ),label='{}'.format(durs[duri]), marker=mrks[duri], lw=0)
    plt.legend()
    plt.show()
    
    nedgeBins_start -= 1
    nedgeBins_end   -= 1


# nedgeBins_start = 6
# nedgeBins_end = 3



In [ ]:
os.listdir('params')

In [ ]:
gv.paramFileRead('params/prep_dat_defparams.ini')

# prep_dat

In [ ]:
%load_ext autoreload
%autoreload 2

#%debug
%run -i run_prep_dat.py --sources_type parcel_aal  --src_grouping_fn 10\
 --src_grouping 0 \
 --rawnames S99_off_hold,S99_off_move,S97_off_move \
 --save_dat 1 --save_stats 1 --param_file prep_dat_defparams.ini

In [ ]:
stats_per_ct

# GenFeatures 

In [ ]:
%load_ext autoreload
%autoreload 2

## Test dataset

In [ ]:
[1, 3].index(33)

In [ ]:
import params

In [ ]:
import io

In [ ]:
#fast until the end
#%debug
%load_ext autoreload
%autoreload 2

%run -i ../run/run_genfeats.py --sources_type parcel_aal --bands crude\
 --src_grouping_fn 10\
 --src_grouping 0 \
 --raw S97_off_move \
 --feat_types con,H_act,H_mob,H_compl,rbcorr\
 --Kalman_smooth 1\
 --load_TFR 0\
 --load_CSD 0\
 --save_TFR 0\
 --save_CSD 0\
 --save_bpcorr 0\
 --save_rbcorr 0\
 --load_rbcorr 0\
 --use_existing_TFR 1\
 --use_preloaded_data 1\
 --load_only 1\
 --show_plots 1\
 --plot_types raw_stats_scatter,feat_stats_scatter\
 --feat_stats_artif_handling no\
 --scale_data_combine_type subj \
 --rbcorr_use_local_means 1\
 --output_subdir test\
 --stats_fn_prefix stats_S97,S99_3_ \
 --param_file genfeats_defparams.ini

In [ ]:
newsfo = subfeature_order_pri[0][:2] + subfeature_order_pri[0][20::30]
newsfo

In [ ]:
extdat_pri[0].shape

In [ ]:
dat_pri[0].shape

In [ ]:
r = utils.makeSimpleRaw(act_pri[0], ch_names=None, sfreq=sfreq)


In [ ]:
%matplotlib qt
r.plot(scalings='auto');

In [ ]:
ivalis

In [ ]:
    raw_perband_flt_pri, raw_perband_bp_pri, chnames_perband_flt_pri, chnames_perband_bp_pri  = \
        ugf.bandFilter(rawnames, times_pri, main_sides_pri, side_switched_pri,
              sfreqs, skips, dat_pri_persfreq, fband_names_inc_HFO, gv.fband_names_HFO_all,
              fbands, n_jobs_flt, subfeature_order, subfeature_order_lfp_hires,
              smoothen_bandpow, ann_MEGartif_prefix_to_use)


In [ ]:
    recalc_stats_multi_band = 1
    if recalc_stats_multi_band:
        means_perband_flt_pri,_ = \
        ugf.gatherMultiBandStats(rawnames,raw_perband_flt_pri, times_pri,
                                chnames_perband_flt_pri, side_switched_pri, sfreq,
                                baseline_int, scale_data_combine_type,
                                artif_handling)
        means_perband_bp_pri,_ = \
        ugf.gatherMultiBandStats(rawnames,raw_perband_bp_pri, times_pri,
                                chnames_perband_bp_pri, side_switched_pri, sfreq,
                                baseline_int, scale_data_combine_type,
                                artif_handling)


In [ ]:
means_perband_flt_pri_ = means_perband_flt_pri
means_perband_bp_pri_ = means_perband_bp_pri

In [ ]:
means_perband_flt_pri

In [ ]:
raw_perband_flt_pri[0]

In [ ]:
fband_names_inc_HFO

In [ ]:
newindsets

In [ ]:
indsetis_valid

In [ ]:
dat_cur.dtype

In [ ]:
type(sfreq_cur)

In [ ]:
opsidelet

In [ ]:
type(low)

In [ ]:
                    r = utils.makeSimpleRaw(dat_cur, ch_names=None, sfreq=sfreq_cur)
                    if bandname.find('HFO') < 0:
                        r.set_annotations(anns_MEGartif)
                        r.set_annotations(anns_LFPartif)
                    else: # we have only LFP channels then
                        r.set_annotations(anns_LFPartif)
                    r.filter(l_freq=low,h_freq=high, n_jobs=n_jobs_flt,
                            skip_by_annotation='BAD_{}'.format(opsidelet), pad='symmetric')#, filter_length="1s" )


In [ ]:
anns_LFPartif[1]

In [ ]:
r[0][0].shape

In [ ]:
                sh = tfrres_cur.shape
                tmp = tfrres_cur.reshape( (sh[0] * sh[1], sh[2] ) ).T
                #scaler.fit( tmp )

                tfr_mean,tfr_std = utsne.robustMean(tmp, axis=-1,ret_std=True,per_dim=1)
                tfrres_cur_ = (tmp - tfr_mean[:,None])/ tfr_std[:,None]
                tfrres_cur_ = tfrres_cur_.reshape(  (sh[0], sh[1], sh[2] ) )


In [ ]:
tfrres_cur_.shape

In [ ]:
anns_pri[0]

In [ ]:
subfeature_order_lfp_hires

In [ ]:
skip

In [ ]:
skip_hires

In [ ]:
skip_div_TFR

In [ ]:
aux_info_perraw['S99_off_hold'][()]

In [ ]:
#fast until the end
#%debug
%load_ext autoreload
%autoreload 2

%run -i run_genfeats.py --sources_type parcel_aal --bands crude\
 --src_grouping_fn 10\
 --src_grouping 0 \
 --raw S99_off_hold,S99_off_move \
 --feat_types con,H_act,H_mob,H_compl\
 --Kalman_smooth 0\
 --load_TFR 1\
 --load_CSD 1\
 --save_TFR 0\
 --save_CSD 0\
 --save_bpcor 0\
 --save_rbcor 0\
 --use_existing_TFR 1\
 --load_only 1\
 --show_plots 1\
 --plot_types raw_stats_scatter,feat_stats_scatter\
 --output_subdir test

In [ ]:
np.zeros((3,3)).ravel()

In [ ]:
X_pri = 

In [ ]:
if plot_feat_stat_scatter:
    int_types_to_stat = [it + '_{}'.format(main_side_let) for it in gp.int_types_basic]
    upre.plotFeatStatsScatter(rawnames,X_pri, int_types_to_stat,
                        feature_names_all,sfreq,
                        times_pri,side_switched_pri, wbd_pri=wbd_H_pri,
                                save_fig=False, separate_by='mod' )
    plt.suptitle('Feat stats before rescaling')
    pdf.savefig()
    plt.close()
    gc.collect()


In [ ]:
len(set(n_channels_pri))

In [ ]:
wdbd.shape

In [ ]:
wbd[0]

In [ ]:
wdbd[0::2]

In [ ]:
np.var( np.diff(wdbd[1::2,:],axis=0) )

In [ ]:
np.diff(wdbd[::2,:],axis=0)

In [ ]:
wdbd

In [ ]:
windowsz/sfreq

In [ ]:
Xtimes[-1]

In [ ]:
times_pri[rawind][-1]

In [ ]:
%load_ext autoreload
%autoreload 2


%run -i run_genfeats.py --sources_type parcel_aal --bands crude\
 --src_grouping_fn 10\
 --src_grouping 0 \
 --raw S99_off_move \
 --feat_types con,H_act,H_mob,H_compl,rbcorr,bpcorr\
 --Kalman_smooth 0\
 --load_TFR 1\
 --load_CSD 1\
 --save_TFR 0\
 --save_CSD 0\
 --save_bpcor 0\
 --save_rbcor 0\
 --use_existing_TFR 1\
 --load_only 1
# --show_plots 1\
# --plot_types feat_timecourse
#--plot_types raw_timecourse
#--plot_types raw_stats,raw_timecourse,raw_psd,feat_stats,feat_timecourse
 #--plot_only 0

In [ ]:
%load_ext autoreload
%autoreload 2

#%debug
%run -i run_genfeats.py --sources_type parcel_aal --bands crude\
 --src_grouping_fn 10\
 --src_grouping 0 \
 --raw S97_off_move \
 --feat_types con,H_act,H_mob,H_compl,rbcorr,bpcorr\
 --Kalman_smooth 0\
 --load_TFR 1\
 --load_CSD 1\
 --save_TFR 0\
 --save_CSD 0\
 --save_bpcor 0\
 --save_rbcor 0\
 --use_existing_TFR 1\
 --load_only 0
# --show_plots 1\
# --plot_types feat_timecourse
#--plot_types raw_timecourse
#--plot_types raw_stats,raw_timecourse,raw_psd,feat_stats,feat_timecourse
 #--plot_only 0

In [ ]:
rawnames

In [ ]:
#search for non-zero components in short msrc names
import re
for nms in nml:
    b = []
    for fen in nms:
    #for fen in feature_names_all:
        r = re.match('.*msrc._[0-9]+_[0-9]+_c[^0][0-9]*', fen)
        if r is not None:
            print(fen)
            b += [fen]
   

In [ ]:
b = []
for fen in bpcor_names:
#for fen in feature_names_all:
    r = re.match('.*msrc._[0-9]+_[0-9]+_c[^0][0-9]*', fen)
    if r is not None:
        print(fen)
        b += [fen]

In [ ]:
scale_data_before_featgen, do_rescale_raws

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#%debug
%run -i run_genfeats.py --sources_type parcel_aal --bands crude\
 --src_grouping_fn 10\
 --src_grouping 0 \
 --raw S99_off_move,S99_off_move \
 --feat_types con,H_act,H_mob,H_compl,rbcorr,bpcorr\
 --Kalman_smooth 0\
 --load_TFR 0\
 --load_CSD 0\
 --save_TFR 0\
 --save_CSD 0\
 --use_existing_TFR 1\
 --load_only 1
# --show_plots 1\
# --plot_types feat_timecourse
#--plot_types raw_timecourse
#--plot_types raw_stats,raw_timecourse,raw_psd,feat_stats,feat_timecourse
 #--plot_only 0

## Double test

In [ ]:
#%debug
%run -i run_genfeats.py --sources_type parcel_aal --bands crude\
 --src_grouping_fn 10\
 --src_grouping 0 \
 --raw S99_off_hold,S99_off_move \
 --feat_types con,H_act,H_mob,H_compl,rbcorr,bpcorr\
 --Kalman_smooth 0\
 --load_TFR 0\
 --load_CSD 0\
 --save_TFR 0\
 --save_CSD 0\
 --use_existing_TFR 1
# --show_plots 1\
# --plot_types feat_timecourse
#--plot_types raw_timecourse
#--plot_types raw_stats,raw_timecourse,raw_psd,feat_stats,feat_timecourse
 #--plot_only 0

## Normal dataset

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#%debug
%run -i run_genfeats.py --sources_type parcel_aal --bands crude\
 --src_grouping_fn 10\
 --src_grouping 0 \
 --raw S01_off_hold,S02_off_hold \
 --feat_types con,H_act,H_mob,H_compl\
 --Kalman_smooth 0\
 --load_TFR 1\
 --load_CSD 0\
 --save_TFR 0\
 --save_CSD 1\
 --use_existing_TFR 1
# --show_plots 1\
# --plot_types feat_timecourse
#--plot_types raw_timecourse
#--plot_types raw_stats,raw_timecourse,raw_psd,feat_stats,feat_timecourse
 #--plot_only 0

In [ ]:
            utsne.concatAnns(rawnames[rawind],times_pri[rawind], artif_mod_str,crop=(crop_start,crop_end),
                        allow_short_intervals=True,
                             side_rev_pri = aux_info_perraw[rawnames[rawind]]['side_switched'],
                             wbd_pri = wbd_H_pri[rawind], sfreq=sfreq)


In [ ]:
times2[-1] + times2[-1]

In [ ]:
wbd_H_pri[rawind][1,-1] / sfreq

In [ ]:
(wbd_H_pri[0][1,-1] + wbd_H_pri[0][1,-1]) / sfreq

In [ ]:
times_pri[0].dtype == float

In [ ]:
ry = raws_permod_both_sides['S01_off_hold']['LFP']

In [ ]:
isinstance(ry, mne.io.BaseRaw)

## Double

In [ ]:
%run -i run_genfeats.py --sources_type parcel_aal --bands crude\
 --src_grouping_fn 10\
 --src_grouping 0 \
 --raw S01_off_hold,S02_off_hold \
 --feat_types con,H_act,H_mob,H_compl\
 --Kalman_smooth 0\
 --load_TFR 1\
 --load_CSD 1\
 --save_TFR 0\
 --save_CSD 0\
 --use_existing_TFR 1\
 --load_only 1 \
 --show_plots 1 --plot_types raw_stats_scatter

In [ ]:
csd_pri

In [ ]:
ntimebins_

In [ ]:
bpow_abscsd_LFP_HFO.shape

In [ ]:
#%debug
%run -i run_genfeats.py --sources_type parcel_aal --bands crude\
 --src_grouping_fn 10\
 --src_grouping 0 \
 --raw S01_off_hold,S02_off_hold \
 --feat_types con,H_act,H_mob,H_compl,rbcorr,bpcorr\
 --Kalman_smooth 0\
 --load_TFR 0\
 --load_CSD 0\
 --save_TFR 0\
 --save_CSD 0\
 --use_existing_TFR 1
# --show_plots 1\
# --plot_types feat_timecourse
#--plot_types raw_timecourse
#--plot_types raw_stats,raw_timecourse,raw_psd,feat_stats,feat_timecourse
 #--plot_only 0

In [ ]:
import utils_preproc as upre
upre.getRawnameListStructure(rawnames=['S01_off_hold','S01_off_move', 
                                       'S01_on_move', 'S03_on_move',
                                       'S03_on_hold']) 

In [ ]:
#%debug
%run -i run_genfeats.py --sources_type parcel_aal --bands crude\
 --src_grouping_fn 10\
 --src_grouping 0 \
 --raw S97_off_move \
 --feat_types con,H_act,H_mob,H_compl\
 --Kalman_smooth 0\
 --load_TFR 0\
 --load_CSD 0\
 --save_TFR 0\
 --save_CSD 0\
 --use_existing_TFR 1\
 --load_only 1
# --show_plots 1\
# --plot_types feat_timecourse
#--plot_types raw_timecourse
#--plot_types raw_stats,raw_timecourse,raw_psd,feat_stats,feat_timecourse
 #--plot_only 0

# runPCA

In [ ]:
%load_ext autoreload
%autoreload 2

## Test dataset 

### Single

In [ ]:
len( f['feature_names_all']  ), len(feature_names_all)

In [ ]:
#roi_labels[gp.src_grouping_names_order[src_file_grouping_ind]] 

In [ ]:
--LFPchan main\

In [ ]:
side_switch_happened_pri

In [ ]:
for i in range(10):
    print(i)
print(i)

In [ ]:
t = [54, 4]
t = t.copy()
#t = t[:]
tt[0] = 222
print(t,tt)

In [ ]:
pwd

In [ ]:
import os,sys
code_dir = os.path.expandvars('$OSCBAGDIS_DATAPROC_CODE')
if code_dir not in sys.path: sys.path.append(code_dir)

In [ ]:
class_labels.shape

In [ ]:
X.shape

In [ ]:
X_for_heavy.shape, class_labels_for_heavy.shape

In [ ]:
    from sklearn.metrics import confusion_matrix


In [ ]:
confusion_matrix([0, 0, 0 ], [23, 3, 3])

In [ ]:
%load_ext autoreload
%autoreload 2
%precision %.4f
#%debug
%run -i ../run/run_ML.py --sources_type parcel_aal --bands_type crude --src_grouping_fn 10\
 --src_grouping 0 \
 --prefix test \
--skip_XGB 0\
--mods LFP\
--allow_CUDA 0\
--subskip_fit 2 \
--n_splits 2 \
--calc_MI 0 \
--save_output 0\
--groupings_to_use merge_movements \
--int_types_to_use basic\
--input_subdir test --output_subdir test\
--raw S97_off_move,S99_off_hold --load_only 0 --show_plots 1\
--plot_types feat_stats --param_file ML_defparams.ini \
--featsel_method XGB_Shapley \
--load_only 1
#--featsel_method SHAP_XGB,interpret_EBM,XGB_Shapley
#--featsel_method interpret_EBM

In [ ]:
%load_ext autoreload
%autoreload 2
%precision %.4f
#%debug
%run -i ../run/run_ML.py --sources_type parcel_aal --bands_type crude --src_grouping_fn 10\
 --src_grouping 0 \
 --prefix test \
--skip_XGB 0\
--mods LFP,msrc\
--LFP_related_only 1\
--cross_couplings_only 1\
--feat_types rbcorr\
--parcel_group_names motor-related\
--allow_CUDA 0\
--subskip_fit 2 \
--n_splits 2 \
--calc_MI 0 \
--save_output 0\
--groupings_to_use merge_movements \
--int_types_to_use basic\
--input_subdir test --output_subdir test\
--raw S97_off_move,S99_off_hold --load_only 0 --show_plots 1\
--param_file ML_defparams.ini \
--featsel_method XGB_Shapley \
--load_only 1

#--featsel_method SHAP_XGB,interpret_EBM,XGB_Shapley
#--featsel_method interpret_EBM

In [ ]:
feature_names_all

In [ ]:
fli = utils.collectFeatTypeInfo(feature_names_all )
print( fli)


In [ ]:
fli

In [ ]:
len( [0] + [None] )

In [ ]:
'Precentral_R'.find(gp.areas_list_aal_my_guess[0])

In [ ]:
features_to_use

In [ ]:
parcel_types

In [ ]:
parcel_group_names

In [ ]:
gp.areas_list_aal_my_guess

In [ ]:
LFP_related_only, 
parcel_group_names, 
remove_corr_self_couplings, 
cross_couplings_only

In [ ]:
fbands_def

In [ ]:
data_modalities

In [ ]:
A = set( range(100) )
print(A)
A - set(range(10))
print(A)
A - set(range(20))
print(A)
A - range(30,40)
print(A)

In [ ]:
ivalis_tb_indarrays_merged.keys()

In [ ]:
revdict

In [ ]:
save_output

In [ ]:
featsel_explainers_res['interpret_EBM']

In [ ]:
y.shape

In [ ]:
X.shape

In [ ]:
y == ebm.predict(X)

In [ ]:
ebm.predict(X)

In [ ]:
class_to_check

In [ ]:
%load_ext autoreload
%autoreload 2
import utils_tSNE as utsne

In [ ]:
sens,spec, F1, confmat  = utsne.getClfPredPower(ebm,X,y,class_to_check, printLog=False)


In [ ]:
perfs_LDA_featsearch[0]

In [ ]:
            res_all_feats = utsne.calcLDAVersions(Xconcat_good_cur, Xconcat_imputed, class_labels_good,
                                  n_components_LDA, class_ind_to_check, revdict,
                                        calcName=lda_version_name,n_splits=n_splits)
            lda_analysis_versions[lda_version_name] = res_all_feats
            gc.collect()


In [ ]:
            res_all_feats = utsne.calcLDAVersions(Xconcat_good_cur, Xconcat_imputed, class_labels_good,
                                  n_components_LDA, class_ind_to_check, revdict,
                                        calcName=lda_version_name,n_splits=n_splits)


In [ ]:


# data = sklearn.datasets.fetch_california_housing()
# # feature_names = ["MedInc", "HouseAge", "AveRooms", "AveBedrms", "Population", "AveOccup",
# #                  "Latitude", "Longitude"]
# X = data.data
# y = data.target
# dmat = xgb.DMatrix(X, y)

In [ ]:
model.__dict__.keys()

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true, y_pred) #, labels=["ant", "bird", "cat"])

In [ ]:
np.mean([np.zeros((3,3)),np.ones((3,3)) ],axis=0)

In [ ]:
help(np.random.randint)

In [ ]:
h = np.max(class_labels_good_for_classif) + 1
confusion_matrix(class_labels_good_for_classif, np.random.randint(0, h, size=len(class_labels_good_for_classif)) )

In [ ]:
confusion_matrix(class_labels_good_for_classif, class_labels_good_for_classif  )

In [ ]:
class_labels_good_for_classif

In [ ]:
#tn, fp, fn, tp = confusion_matrix([0, 1, 0, 1], [1, 1, 1, 0]).ravel()

In [ ]:
XGB_max_weight=4

In [ ]:

add_clf_creopts={ 'n_jobs':n_jobs_XGB, 'use_label_encoder':False,
                 'importance_type': 'total_gain', 'max_depth':XGB_max_weight }
tree_method = XGB_tree_method
method_params = {'tree_method': tree_method}

if (XGB_tree_method in ['hist', 'gpu_hist']) \
        and allow_CUDA \
        and len(GPUs_list):
    tree_method = 'gpu_hist'

    method_params['gpu_id'] = GPUs_list[0]

add_clf_creopts.update(method_params)
clf_XGB = XGBClassifier(**add_clf_creopts)
# fit the clf_XGB to get feature importances
print('Starting XGB on X.shape ', X_for_heavy.shape)
add_fitopts = { 'eval_metric':'logloss'}
clf_XGB.fit(X_for_heavy, class_labels_good_for_classif, **add_fitopts)


In [ ]:
clf_XGB.max_depth

In [ ]:
clf_XGB.n_classes_

In [ ]:
clf_XGB._Booster.__dict__.keys()

In [ ]:
clf_XGB.__dict__.keys()

In [ ]:
grouping_key

In [ ]:
GPUtil.showUtilization()

In [ ]:
import xgboost as xgb

X = X_for_heavy
y = class_labels_good_for_classif
dmat = xgb.DMatrix(X, y)



for each pair of classes 
throw away datapoints with other labels
perform classif

In [ ]:
class_labels_for_heavy

## EBM

### test

In [ ]:
import interpret

In [ ]:
from interpret.glassbox import ExplainableBoostingClassifier
seed = 0
ebm = ExplainableBoostingClassifier(random_state=seed, feature_names=featnames_nice)
ebm.fit(X, y)
global_exp = ebm.explain_global()

### 2

In [ ]:
clu = list(set(class_labels_for_heavy))
print(clu)
le.transform(clu)

In [ ]:
revdict

In [ ]:
#                class_labels, class_labels_good, revdict, class_ids_grouped


In [ ]:
EBM_result_per_cp= {}
indpairs_names = []

import itertools
# since EBM only works for binary, I treat each pair of classes separately
uls = list(set(class_labels_for_heavy))
class_pairs = list(itertools.combinations(uls, 2))
print(class_pairs)

#cpi = 0
for cpi in range(len(class_pairs)):
    c1,c2 = class_pairs[cpi]
    inds = np.where( (class_labels_good_for_classif == c1) | (class_labels_good_for_classif == c2)  )[0]
    #inds2 = np.where(class_labels_good_for_classif == c2)[0]

    ipo = le.inverse_transform([c1,c2])
    indpair_names = ( revdict[ipo[0]], revdict[ipo[1]] )
    
    print(f'Starting computing EBM for class pair {indpair_names}, in total {len(inds)}'+
          f'=({sum(class_labels_good_for_classif == c1)}+{sum(class_labels_good_for_classif == c2)}) data points')

    # filter classes
    X = Xconcat_good_cur[inds]
    y = class_labels_good_for_classif[inds]

    import interpret
    from interpret.glassbox import ExplainableBoostingClassifier
    seed = 0
    ebm = ExplainableBoostingClassifier(random_state=seed, feature_names=featnames_nice, n_jobs=n_jobs_XGB)
    ebm.fit(X, y)
    global_exp = ebm.explain_global()

    EBM_result_per_cp[indpair_names] = global_exp
    indpairs_names += [indpair_names]
    
    # extracting data from explainer
    scores = global_exp.data()['scores']
    names = global_exp.data()['names']
    sis = np.argsort(scores)[::-1]
    featnames_srt = np.array(names)[sis]   
    print(f'Strongest is {featnames_srt[0]}')
    # TODO: use classs_names

In [ ]:
print(indpairs_names[0])
interpret.show(EBM_result_per_cp[indpair_names])

In [ ]:
global_exp.data(1)

In [ ]:
global_exp.__dict__.keys()

In [ ]:
global_exp.explanation_type

In [ ]:
global_exp._internal_obj['overall']['scores']

In [ ]:
import xgboost as xgb
dmat = xgb.DMatrix(X, y)

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
#Shapley

In [ ]:
set(y)

In [ ]:
import inspect
lines = inspect.getsource(bst.predict)
print(lines)

In [ ]:
help(bst.predict)

#pred_contribs (bool) – When this is True the output will be a matrix of size (nsample, nfeats + 1) with each record indicating the feature contributions (SHAP values) for that prediction. The sum of all feature contributions is equal to the raw untransformed margin value of the prediction. Note the final column is the bias term.

In [ ]:
#%%timeit -n1 -r1
clf_XGB2 = XGBClassifier(**add_clf_creopts)
clf_XGB2.fit(X, y, **add_fitopts)


bst = clf_XGB2.get_booster()
bst.set_param({"predictor": "gpu_predictor"})
shap_values = bst.predict(dmat, pred_contribs=True)
shap_values.shape
#shap_interaction_values = bst.predict(dmat, pred_interactions=True)

In [ ]:
%matplotlib inline

In [ ]:
def plot_feature_importance(feature_names, shap_values):
 # Get the mean absolute contribution for each feature
 aggregate = np.mean(np.abs(shap_values[:, 0:-1]), axis=0)
 # sort by magnitude
 z = [(x, y) for y, x in sorted(zip(aggregate, feature_names), reverse=True)]
 z = list(zip(*z))
 plt.bar(z[0], z[1])
 plt.xticks(rotation=90)
 plt.tight_layout()
 plt.show()

plt.title('Shap values from XGB package');
plot_feature_importance(featnames_nice, shap_values)


In [ ]:
clf_XGB2.feature_importances_

In [ ]:
def plot_feature_importance0(feature_names, importances):
 # Get the mean absolute contribution for each feature
 #aggregate = np.mean(np.abs(shap_values[:, 0:-1]), axis=0)
 # sort by magnitude
 z = [(x, y) for y, x in sorted(zip(importances, feature_names), reverse=True)]
 z = list(zip(*z))
 plt.bar(z[0], z[1])
 plt.xticks(rotation=90)
 plt.tight_layout()
 plt.show()

plt.title('Feature importances from XGB package');
plot_feature_importance0(featnames_nice, clf_XGB2.feature_importances_)

In [ ]:
X.shape

In [ ]:
clf_XGB2.inplace_predict()

In [ ]:
clf_XGB3 = XGBClassifier(**add_clf_creopts)
clf_XGB3.fit(X, y, **add_fitopts)


bst = clf_XGB2.get_booster()
bst.set_param({"predictor": "cpu_predictor"})

# shap package

nsamples = max(50, X.shape[0] // 10 )
Xsubset = shap.utils.sample(X, nsamples)


explainer= shap.Explainer(clf_XGB2.predict, Xsubset, feature_names=featnames_nice)
#explainer= shap.Explainer(clf_bestfeats.predict, Xsubset)

shap_values3 = explainer(X )

In [ ]:
plt.title('Shap values from shap package');
plot_feature_importance(featnames_nice, shap_values2.values)

In [ ]:
# GPU -- I need to restart ipython perhaps because cupy wants some env var to be initialized
import cupy as cp
X_gpu = cp.array(X)

# shap package

nsamples = max(50, X.shape[0] // 10 )
Xsubset = shap.utils.sample(X, nsamples)


explainer= shap.Explainer(clf_XGB2.predict, Xsubset, feature_names=featnames_nice)
#explainer= shap.Explainer(clf_bestfeats.predict, Xsubset)

shap_values2 = explainer(X_gpu )


In [ ]:
shap_values.shape

In [ ]:
help(bst.predict)

In [ ]:
shap_values.shape

In [ ]:
shap_interaction_values.shape

In [ ]:
? %timeit

In [ ]:
%%timeit -n1 -r1
bst = model.get_booster()
bst.set_param({"predictor": "cpu_predictor"})
shap_values = bst.predict(dmat, pred_contribs=True)

In [ ]:
                cuda_par = {'tree_method': 'gpu_hist'}
                if mne.utils.get_config('MNE_USE_CUDA'):
                    add_clf_creopts.update(cuda_par)
                model = XGBClassifier(**add_clf_creopts)
                # fit the model to get feature importances
                print('Starting XGB on X.shape ', X_for_heavy.shape)
                add_fitopts = { 'eval_metric':'logloss'}
                model.fit(X_for_heavy, class_labels_good_for_classif, **add_fitopts)


In [ ]:
                add_clf_creopts={ 'n_jobs':n_jobs_XGB, 'use_label_encoder':False }

                model = XGBClassifier(**add_clf_creopts)
                # fit the model to get feature importances
                print('Starting XGB on X.shape ', X_for_heavy.shape)
                add_fitopts = { 'eval_metric':'logloss'}
                model.fit(X_for_heavy, class_labels_good_for_classif, **add_fitopts)

In [ ]:
shap_values

In [ ]:
#%debug
%run -i run_PCA.py --sources_type parcel_aal --bands_type crude --src_grouping_fn 10\
 --src_grouping 0 \
 --prefix test \
--skip_XGB 0\
--mods msrc\
--subskip_fit 8 \
--input_subdir test --output_subdir test\
--bands beta\
--raw S99_off_move,S99_off_hold --load_only 0 --save_output 1\
 --self_couplings_only 1\
--show_plots 1 --plot_types feat_stats

In [ ]:
help(plt.subplots)

In [ ]:
X_pri[0].shape

In [ ]:
featnames

In [ ]:
len(featnames)

In [ ]:
do_outliers_discard, artifact_handling, remove_unlabeled

In [ ]:
len(bininds_concat_good)

In [ ]:
len(np.where( mask[sl] )[0] )

In [ ]:
len(bininds_good_cur)

In [ ]:
#%debug
%run -i run_PCA.py --sources_type parcel_aal --bands_type crude --src_grouping_fn 10\
 --src_grouping 0 \
 --prefix test \
--skip_XGB 0\
--subskip_fit 8 \
--parcel_types Cerebellum \
--input_subdir test --output_subdir test\
--raw S99_off_move --load_only 1 --save_output 1

In [ ]:
#%debug
%run -i run_PCA.py --sources_type parcel_aal --bands_type crude --src_grouping_fn 10\
 --src_grouping 0 \
 --prefix test \
--skip_XGB 0\
--subskip_fit 8 \
--parcel_types Cerebellum \
--raw S97_off_move,S99_off_move --load_only 1 --save_output 0

In [ ]:
stds

In [ ]:
means

In [ ]:
#search for non-zero components in short msrc names
for fen in featnames:
    r = re.match('.*msrc._[0-9]+_[0-9]+_c1[0-9]*', fen)
    if r is not None:
        print(fen)
        
#search for non-zero components in long msrc names
for fen in featnames_nice:
    r = re.match('.*msrc_.*_c1[0-9]*', fen)
    if r is not None:
        print(fen)

In [ ]:
featnames_nice

### Double

In [ ]:
%load_ext autoreload
%autoreload 2

%run -i run_PCA.py --sources_type parcel_aal --bands_type crude --src_grouping_fn 10\
 --src_grouping 0 \
 --prefix test \
--subskip_fit 4\
--groupings_to_use merge_all_not_trem,merge_nothing\
--int_types_to_use basic,trem_vs_quiet\
--raw  S99_off_hold,S97_off_move --load_only 1 --show_plots 1 --plot_types feat_stats

In [ ]:
grouping, sides_hand

In [ ]:
    if show_plots and 'feat_stats' in plot_types:
        int_types_to_stat = [it + '_{}'.format(main_side_let) for it in gp.int_types_basic]
        upre.plotFeatStatsScatter(rawnames, X_pri, int_types_to_stat,
                            featnames,sfreq,
                            rawtimes_pri,side_switch_happened_pri, wbd_pri=wbd_pri,
                                    save_fig=True, separate_by = 'feat_type' )
        #plt.suptitle('Feat stats before rescaling')
        #pdf.savefig()
        #plt.close()
        #gc.collect()


In [ ]:
featnames[0]

In [ ]:
re.match("([a-zA-Z]+)_.*",featnames[0]).groups()[0]


In [ ]:
class_ids_grouped

In [ ]:
revdict

In [ ]:
counts = utsne.countClassLabels(class_labels_good, revdict=revdict) 
counts

In [ ]:
ivalis_tb_indarrays_merged.keys()

In [ ]:
new_main_side

In [ ]:
class_ids_grouped

In [ ]:
class_labels == 0

In [ ]:
set(class_labels)

## four

In [ ]:
%load_ext autoreload
%autoreload 2
#%debug
%run -i run_PCA.py --sources_type parcel_aal --bands_type crude --src_grouping_fn 10 \
 --src_grouping 0 \
 --prefix test \
 --LFP_related_only 1\
 --raw S99_off_hold,S99_off_move,S99_on_hold,S99_on_move\
 --groupings_to_use merge_nothing,merge_within_subj,merge_within_medcond,merge_within_task\
--int_types_to_use basic,trem_vs_quiet,subj_medcond_task,subj_medcond,subj 

## Normal dataset 

In [ ]:
%load_ext autoreload
%autoreload 2

### Single

In [ ]:
#%debug
%run -i run_PCA.py --sources_type parcel_aal --bands_type crude --src_grouping_fn 10\
 --src_grouping 0 \
 --prefix test \
--raw S01_off_hold --load_only 0

### Double

In [ ]:
#%debug
%run -i run_PCA.py --sources_type parcel_aal --bands_type crude --src_grouping_fn 10\
 --src_grouping 0 \
 --groupings_to_use merge_nothing,merge_within_subj,merge_within_medcond,merge_within_task\
--int_types_to_use basic,trem_vs_quiet,subj_medcond_task,subj_medcond,subj \
--prefix test --LFP_related_only 1\
--raw S01_off_hold,S01_off_move \
--show_plots 1 --plot_types feat_stats --load_only 1

In [ ]:
set(feature_names_pri[0]) - set(good_feats) 

In [ ]:
set(good_feats) - set(feature_names_pri[0])

In [ ]:
feature_names_pri

In [ ]:
new_main_side_pri

In [ ]:
output_subdir

In [ ]:
anns, anns_pri, times_concat, dataset_bounds, wbd_merged = utsne.concatAnns(rawnames,
                                                          rawtimes_pri, crop=(crop_start,crop_end),
                                                          side_rev_pri = side_switch_happened_pri,
                                                         wbd_pri = wbd_pri, sfreq=sfreq, ret_wbd_merged=1)


In [ ]:
wbd_merged[1,-1] / sfreq, (times_concat[-1] + 1/sfreq)

In [ ]:
wbd_merged[:,0]

In [ ]:
d = np.diff(times_concat)
min(d),max(d), np.mean(d)

In [ ]:
np.sum(d>0.005)

In [ ]:
wbd

In [ ]:


    

plotFeatStatsScatter(rawnames,X_pri_pre, ['notrem_L', 'trem_L', 'entire'],
                         feature_names_pri,wbd_pri,sfreq,
                         rawtimes_pri,side_switch_happened_pri)


In [ ]:
plotFeatStatsScatter(rawnames,X_pri, ['notrem_L', 'trem_L', 'entire'],
                         feature_names_pri,wbd_pri,sfreq,
                         rawtimes_pri,side_switch_happened_pri)

In [ ]:
f.get('wbd',None)

In [ ]:
#%debug
%run -i run_PCA.py --sources_type parcel_aal --bands_type crude --src_grouping_fn 10\
 --src_grouping 0 \
 --prefix test --LFP_related_only 1\
--raw S01_off_hold,S02_off_hold --load_only 0

In [ ]:
lda_analysis_versions['all_present_features']['CV'].keys()

In [ ]:
LFP_related_only

In [ ]:
import datetime

In [ ]:
np.arange(4)

In [ ]:
np.logical_and(np.ones(4),np.ones(4),np.arange(4).astype(bool))

In [ ]:
wbd_pri[0].shape

In [ ]:
X.shape

In [ ]:
#%debug
Xs = upre.rescaleFeats(rawnames, Xs, feature_names_pri[0], wbd_pri,
                 sfreq, rawtimes_pri, int_type_pri = int_type_pri,
                 minlen_bins = 5 * 256 / 32, combine_within='no')


In [ ]:
from sklearn.model_selection import cross_validate

In [ ]:
cross_validate

In [ ]:
def myfun(y_true,y_pred):
    span = (np.max(y_true) - np.min(y_true))
    #print(span)
    assert span <= 1
    # I need to give y_true with only two possible values 
    #print(y_true,y_pred)
    ntot = len(y_true)
    TP = sum(y_true == y_pred)
    FP = sum(y_true != y_pred)
    return len(y_true)

In [ ]:
from sklearn.metrics import make_scorer

In [ ]:
# scoring = {'prec_macro': 'precision_macro',
#            'rec_macro': make_scorer(recall_score, average='macro')}

scoring = ['balanced_accuracy', 'precision', 'recall', 'f1']
scoring = {'specificity':make_scorer(myfun)}

In [ ]:
# add groups corresponding to patiend & condition numbmer. Maybe I can use rawnames directly

In [ ]:
cross_validate(model, Xconcat_good_cur, class_labels_good, scoring=scoring, 
               cv=5, return_train_score=True)

In [ ]:
import sklearn
help(sklearn.metrics)

In [ ]:
help(cross_validate)

In [ ]:
len(parcel_types),parcel_types[0]

In [ ]:
src_rec_info_pri[0]

In [ ]:
#roi_labels_cur = roi_labels[srcgrouping_names_sorted[0]]

# Kalman

In [ ]:
rawlfp = mne.io.read_raw( os.path.join(gv.data_dir,"S07_on_move_LFPonly.fif") )

In [ ]:
rawlfp.load_data()

#utils.

#rawlfp.filter(l_freq=10, h_freq=30)

chdata = rawlfp.get_data()

chd = chdata[0]

chd_ang, chd_instfreq, chd_ampl = utils.getBandHilbDat(chd,sfreq,10,30)



plt.figure(figsize=(14,4))
plt.plot(chd_ampl[:5*sfreq])

inds = slice(None,10*sfreq)
#inds = slice(None,None)
chd_test = chd_ampl[inds]
times_test = rawlfp.times[inds]

In [ ]:
estim = chd_test[:300]

In [ ]:
span = np.max(chd_test ) - np.min(chd_test)

In [ ]:
%%latex
\begin{gather}
x_k = A x_{k-1} + q_{k-1}, \qquad q_{k-1} \sim N(0, Q) \\
y_k = H x_k + r_k, \qquad r_k \sim N(0, R),
\end{gather}

In [ ]:
procnoise_coef = 0.05
noisecov_coef = 200
kf = simdkalman.KalmanFilter(
    state_transition = [[1]],        # matrix A
    process_noise = np.diag([span*procnoise_coef]),    # Q
    observation_model = np.array([[1]]),   # H
    observation_noise = np.std(estim)*noisecov_coef)   # R
# to smoothen I have to have small process noise and large observation noise


smoothed = kf.smooth(chd_test,
                     initial_value = [np.mean(estim)],
                     initial_covariance = np.diag([np.std(estim)]))

plt.figure(figsize=(14,4))
plt.plot(times_test,chd_test)
plt.plot(times_test,smoothed.states.mean[:,0])

In [ ]:
help(LowessSmoother)

In [ ]:
from tsmoothie.smoother import LowessSmoother
smoother = LowessSmoother(smooth_fraction=0.5, iterations=1)
smoother.smooth(chd_test)

plt.figure(figsize=(14,4))
plt.plot(times_test,chd_test)
plt.plot(times_test,smoother.data[0])

In [ ]:
#help(simdkalman)

In [ ]:
chdata.shape

In [ ]:
%matplotlib inline

In [ ]:
smoothed_.shape

In [ ]:
import multiprocessing as mpr
n_jobs = None
if n_jobs is None:
    n_jobs = max(1, mpr.cpu_count() - 2 )
elif n_jobs == -1:
    n_jobs = mpr.cpu_count()

In [ ]:
estim = chd_test[:300]

chd_test2 = np.vstack([chd_test, chd_test])
sz = 1e-2
chd_test2[1] += np.random.uniform(low=-sz,high=sz,size=(len(chd_test)))

Tp = 1 * sfreq  # the smaller the smoother
estim = chd_test2[:,:300]

n_jobs = 1
smoothed_ = smoothData(chd_test2,Tp,estim, n_jobs=n_jobs)

for chi in range(smoothed_.shape[0]):
    plt.figure(figsize=(14,4))
    plt.plot(times_test,chd_test,label='test')
    #plt.plot(times_test,smoothed.states.mean[:,0])
    plt.plot(times_test,smoothed_[chi],label='smoothed {}'.format(chi))
    plt.legend()
    plt.show()

In [ ]:
smoothed_.shape

In [ ]:
chd_test2 = np.vstack([chd_test, chd_test])

In [ ]:
import gc; gc.collect()

In [ ]:
chd_test2.shape

In [ ]:
ar = np.array( [[0, np.nan, 0, 1 , 3], [44, 5, 0, 1 , 3], [4, 4, 5 ,np.nan, 54]] )
display(ar)
#ar = ar.T
print(ar.shape)
np.where( np.isnan(ar) )

In [ ]:
smoothed_ = smoothData(chdata,chdata[:,:sfreq*5],Tp)
plt.figure(figsize=(14,4))
plt.plot(rawlfp.times,chdata[0])
#plt.plot(times_test,smoothed.states.mean[:,0])
plt.plot(rawlfp.times,smoothed_[0])
plt.xlim(0,10)

In [ ]:
smoothed_.shape

In [ ]:
smoothed_[0]

In [ ]:
smoothed.states

In [ ]:
plt.plot(smoothed.states.cov[10:-10,0])
#plt.plot(smoothed.states.cov[10:-10,1])

In [ ]:
import tsmoothie

In [ ]:
help(tsmoothie.smoother.KalmanSmoother)

In [ ]:
%debug
%run -i run_PCA.py --sources_type parcel_aal --bands_type crude --src_grouping_fn 9\
 --src_grouping 0 \
 --prefix onlyMotorSrc_test \
--parcel_types motor-related --crop 170,220 --raw S01_off_hold

In [ ]:
bad_parcel_inds

In [ ]:
good_feats

In [ ]:
crop_end

In [ ]:
len()

In [ ]:
all_parcels

In [ ]:
regexes

In [ ]:
len(inds_bad_parcels)

In [ ]:
parcel_types

In [ ]:
bad_parcels

In [ ]:
roi_labels

# tSNE

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#%debug
%run -i run_tSNE.py --sources_type parcel_aal --src_grouping_fn 10\
 --src_grouping 0 \
 --prefix test \
--subskip 8 --output_subdir test_S01\
--input_subdir test\
--raw S99_off_hold,S99_off_move --load_only 1

In [ ]:
# import matplotlib
# matplotlib.use('Qt4Agg')
#['GTK3Agg', 'GTK3Cairo', 'MacOSX', 'nbAgg', 'Qt4Agg', 'Qt4Cairo', 
#'Qt5Agg', 'Qt5Cairo', 'TkAgg', 'TkCairo', 'WebAgg', 'WX', 
#'WXAgg', 'WXCairo', 'agg', 'cairo', 'pdf', 'pgf', 'ps', 'svg', 'template']


In [ ]:
%matplotlib 

In [ ]:
%load_ext autoreload
%autoreload 2

#%debug
%run -i run_tSNE.py --sources_type parcel_aal --src_grouping_fn 10\
 --src_grouping 0 \
 --prefix test \
--subskip 8 --output_subdir test_S01\
--raw S01_off_hold,S01_off_move --load_only 1

# Plotting best feats

In [ ]:
sind_str,mc,tk  = utils.getParamsFromRawname(rawnames[0])
sources_type='parcel_aal'
src_file_grouping_ind = 10
src_rec_info_fn = '{}_{}_grp{}_src_rec_info'.format(rawname_,
                                                    sources_type,src_file_grouping_ind)
src_rec_info_fn_full = os.path.join(gv.data_dir, src_rec_info_fn + '.npz')
rec_info = np.load(src_rec_info_fn_full, allow_pickle=True)


print( list(rec_info.keys()) )

labels_dict = rec_info['label_groups_dict'][()]
srcgroups_dict = rec_info['srcgroups_dict'][()]
coords = rec_info['coords_Jan_actual'][()]
srcgrouping_names_sorted = rec_info['srcgroups_key_order'][()]


In [ ]:
sgdn = 'all_raw'
clrs = utils.vizGroup(sind_str,coords,labels_dict[sgdn],srcgroups_dict[sgdn], show=False)

import mayavi.mlab as mam
#x,z axis views


# mam.pitch(0)
# mam.roll(0)
#mam.view(0,0,'auto','auto',0,0)  #Z axis, from above
#mam.view(90,0,'auto','auto',0,0)  #Z axis, from above

#mam.view(6,2,'auto','auto')
#mam.view(0,0)  # Z above
mam.view(0,90)  # Y

mam.orientation_axes()
mam.show()

In [ ]:
mam.close()

In [ ]:
lda_output.keys()

In [ ]:
best_feats_dict = {}

feat_info_pri

featnames_nice = utils.nicenFeatNames(featnames_filtered,
                                    labels_dict,srcgrouping_names_sorted)
featnames_nice = np.array(featnames_nice)



if 'MI_per_feati' in lda_output:
    MI = lda_output['MI_per_feati']

    best_featsMI_incr = np.argsort(MI)
    nbest_MI = 10

    strong_inds = best_featsMI_incr[-nbest_MI:]    
    best_feats_dict['MI'] = strong_inds

strong_inds = lda_output['strong_inds_XGB']
best_feats_dict['XGB'] = strong_inds

strong_inds = lda_output['inds_important']
best_feats_dict['LDA_naive'] = strong_inds

# pca_derived_featinds_perthr = lda_output['pca_derived_featinds_perthr']
# feat_variance_q_thr = lda_output['feat_variance_q_thr']

# for thri,thr in feat_variance_q_thr:
#     best_feats_dict['PCA_{:.2f}'.format(thr)] = pca_derived_featinds_perthr[thri] 

In [ ]:
bft = 'LDA_naive'
#bft = 'XGB'
strong_inds = best_feats_dict[bft]
#feat_names_cur = featnames_nice[strong_inds]
feat_names_cur = featnames_filtered[strong_inds]


In [ ]:
import re
src_chns_per_feat_nice = []
src_chns_all_nice = []
parcel_indices = []
parcel_indices_all = []
for fni,fn in enumerate(feat_names_cur):
    p = 'msrc._[0-9]+_[0-9]+_c[0-9]+'
    source_chns_cur_featname = re.findall(p, fn)
    
    sides,groupis,parcelis,compis = utils.parseMEGsrcChnamesShortList(source_chns_cur_featname)
    parcel_indices_all += parcelis
    parcel_indices += [parcelis]
    
    tmp = list(srcgrouping_names_sorted) * 10  # because we have 9 there
    nice_chns_cur_featname = utils.nicenMEGsrc_chnames(source_chns_cur_featname,labels_dict,tmp,
                            prefix='msrc_')
    nice_chns_cur_featname = list(set(nice_chns_cur_featname))
    src_chns_per_feat_nice += [nice_chns_cur_featname]
    src_chns_all_nice += nice_chns_cur_featname
    print(fn,nice_chns_cur_featname)

sgdn='all_raw'
parcel_indices_all = list(set(parcel_indices_all))
#parcel_indices_all

roi_labels = np.array(  labels_dict[sgdn] )
rls = ['unlabeled'] + list( roi_labels[parcel_indices_all] )

srcgrp = np.zeros( srcgroups_dict[sgdn].shape, dtype=srcgroups_dict[sgdn].dtype)

for pii,pi in enumerate(parcel_indices_all):
    srcgrp[srcgroups_dict[sgdn] == pi] = pii + 1 #list(roi_labels).index( rls[pii])


In [ ]:
%matplotlib qt

In [ ]:


clrs =  vizGroup2(sind_str,coords,rls,srcgrp, show=False, alpha=.1)



plt.tight_layout()
plt.savefig('{}_srcmap_top.png'.format(bft))

In [ ]:
help(plt.scatter)

In [ ]:
# utils.vizGroup(sind_str,coords,rls,srcgrp, show=False)

# import mayavi.mlab as mam

# mam.view(0,0)  # Z
# mam.savefig('{}_srcmap_top.png'.format(bft), size=(500,500))
# mam.view(0,90)  # Y,  right side is closer to the viewer
# mam.orientation_axes()
# mam.savefig('{}_srcmap_sideR.png'.format(bft), size=(500,500))

# mam.show()
# mam.close()

In [ ]:
a = (3,4)
a,4

In [ ]:
mam.close()

In [ ]:
strong_inds

In [ ]:
best_featn

In [ ]:
lda_anver.keys()

In [ ]:
node_names

In [ ]:
con.dtype

In [ ]:
%matplotlib inline

In [ ]:
con = np.arange(4)/50
node_names = list( map(str, np.arange(4)) )
tuplinds=( [1, 2, 3, 0],[3, 1, 1, 1])
tuplinds=( np.array(tuplinds[0]), np.array(tuplinds[1]))

mne.viz.plot_connectivity_circle(con, node_names, indices=tuplinds, n_lines=None,
                                title='fddd')
#plt.suptitle('fds')

    con : array
        Connectivity scores. Can be a square matrix, or a 1D array. If a 1D
        array is provided, "indices" has to be used to define the connection
        indices.
            node_names : list of str
        Node names. The order corresponds to the order in con.
    indices : tuple of array | None
        Two arrays with indices of connections for which the connections
        strengths are defined in con. Only needed if con is a 1D array.

In [ ]:
help(mne.viz.plot_connectivity_circle)


In [ ]:
subskip

In [ ]:
X_cur_lda_type_[0].shape

In [ ]:
lda_anver.keys()

In [ ]:
len(tSNE_result)

In [ ]:
out_name

In [ ]:
fig_fname

In [ ]:
if show_plots and do_tSNE:
    print('Starting tSNE plotting ')
    print('!!! only zero seed !! ')
    if not os.path.exists(os.path.join(gv.dir_fig, output_subdir) ):
        os.makedirs(os.path.join(gv.dir_fig, output_subdir) )
    pdf= PdfPages( os.path.join(gv.dir_fig, output_subdir, out_name+'.pdf' ) )

    #cols = [colors, colors2, colors3]
    cols = [colors]

    colind = 0
    nr = len(points_type) * len(dim_red_algs )
    nc = len(perplex_values)
    ww = 8; hh=8
    fig,axs = plt.subplots(ncols =nc, nrows=nr, figsize = (nc*ww, nr*hh))
    if not isinstance(axs,np.ndarray):
        axs = np.array([[axs]] )
    # for pi,pv in enumerate(perplex_values):
    #     for si,sv in enumerate(seeds):
    for tpl in tSNE_result:
        #for algi in range(len(dim_red_algs)):
        pi,si,X_embedded, seed, param, lrate, dtype, alg = tpl
        algi = dim_red_algs.index(alg)
        rowind = points_type.index(dtype )
        ax = axs[si + len(seeds)*algi,pi ]
        #X_embedded = res[si][pi]
        #ax.scatter(X_embedded[:,0], X_embedded[:,1], c = cols[colind], s=5)

        utsne.plotMultiMarker(ax,X_embedded[:,0], X_embedded[:,1], c = cols[colind], s=5,
                                m=markers)
        ax.set_title('{} from {} param = {};\nlrate = {}; seed = {}'.
                    format(alg, dtype, param, lrate, seed))

    axs[0,0].legend(handles=legend_elements)
    #figname = 'tSNE_inpdim{}_PCAdim{}_Npts{}_hside{}_skip{}_wsz{}_lrate{}.pdf'.\
    #    format(n_feats, pcapts.shape[-1], X_PCA.shape, mts_letter, totskip, windowsz, lrate)


    str_feats = ','.join(PCA_info['features_to_use'])
    str_mods = ','.join(PCA_info['data_modalities'])
    out_name += '\nmainLFP{}_HFO{}_{}_{}\nXshape={}'.\
        format(int(PCA_info['use_main_LFP_chan']), int(PCA_info['use_lfp_HFO']),
               str_mods, str_feats, X_PCA.shape)

    figname = out_name + '.pdf'
    plt.suptitle(figname)
    #plt.savefig(rawname_ +'_'+figname)

    pdf.savefig()
    pdf.close()


In [ ]:
os.path.join(gv.dir_fig, output_subdir, out_name+'.pdf' )

In [ ]:
figname

In [ ]:
ldapts.shape

In [ ]:
siXGB

In [ ]:
feat_info_pri[0].keys()

In [ ]:
rec_info['labels_dict']

In [ ]:
import globvars as gv
import os
import mne,utils

In [ ]:
#anns_cnv, ivalis_perside = utils.intervalJSON2Anns(rawname_)

In [ ]:
# get info about bad MEG channels (from separate file)
with open('subj_info.json') as info_json:
        #raise TypeError

    #json.dumps({'value': numpy.int64(42)}, default=convert)
    gen_subj_info = json.load(info_json)

In [ ]:
from globvars import gp

In [ ]:
#gv.gen_subj_info
import json

In [ ]:

rawname_ = 'S01_off_hold'
lens =upre.getGenIntervalInfoFromRawname(rawname_);#,interval=(150,250));
lens_short = upre.getGenIntervalInfoFromRawname(rawname_,interval=(170,220));

In [ ]:
list(sorted(lens.keys()))[0].find('_L')

In [ ]:
rawnames =   ['S01_off_hold', 'S01_off_move'] + ['S01_on_hold', 'S01_on_move']
rawnames +=  ['S02_off_hold', 'S02_off_move'] + ['S02_on_hold', 'S02_on_move'] 
rawnames +=  ['S03_off_hold', 'S03_off_move']
rawnames +=  ['S04_off_hold', 'S04_off_move'] + ['S04_on_hold', 'S04_on_move'] 
rawnames +=  ['S05_off_hold', 'S05_off_move'] + ['S05_on_hold', 'S05_on_move'] 
rawnames +=  ['S06_off_hold', 'S07_off_move'] + ['S06_on_hold', 'S07_on_move'] 
rawnames +=  ['S07_off_hold', 'S07_off_move'] + ['S07_on_hold', 'S07_on_move'] 
rawnames +=  ['S08_off_hold', 'S07_off_move'] + ['S08_on_hold', 'S07_on_move'] 
rawnames +=  ['S09_off_hold', 'S07_off_move'] + ['S09_on_hold', 'S07_on_move'] 
rawnames +=  ['S10_off_hold', 'S07_off_move'] + ['S10_on_hold', 'S07_on_move'] 

In [ ]:
%%capture cap

for rawname_ in rawnames:
    try:
        getGenIntervalInfoFromRawname(rawname_);
    except FileNotFoundError as fnfe:
        print(fnfe)
        
with open('interval_stats_output.txt', 'w') as f:
    f.write(cap.stdout)

        


# Resave

In [ ]:
#%debug
%run -i resave.py

In [ ]:
anns

In [ ]:
anns[0]

In [ ]:
%run -i run_genfeats.py --sources_type parcel_aal --bands crude\
 --src_grouping_fn 10\
 --src_grouping 0 \
 --raw S01_off_move,S02_off_move \
 --feat_types con,H_act,H_mob,H_compl 

In [ ]:
%run -i run_PCA.py --prefix all --rawnames S01_off_move,S02_off_move --sources_type parcel_aal \
--skip_XGB 0 --bands_type crude --src_grouping 0 --src_grouping_fn 10 --groupings_to_use merge_nothing \
--int_types_to_use basic,trem_vs_quiet --show_plots 0

In [ ]:
#mne.viz.plot_ica_properties(ica,filt_raw,[0,1], show=0, psd_args={'fmax':75} ) 

In [ ]:
%run -i run_PCA.py --prefix all --rawnames S01_off_hold,S01_off_move --sources_type parcel_aal \
--skip_XGB 1 --bands_type crude --src_grouping 0 --src_grouping_fn 9 --groupings_to_use merge_nothing \
--int_types_to_use basic,trem_vs_quiet --show_plots 0

In [ ]:
ldas_CV = lda_output_pg['merge_nothing']['basic']['ldaobjs_CV']

In [ ]:
ldas_CV[0].__dict__.keys()

In [ ]:
            scalings_list = [lda_cur.scalings_ for lda_cur in ldas_CV   ]
            xbars_list     = [lda_cur.xbar_     for lda_cur in ldas_CV   ]
            rotations_list = [lda_cur.rotations_  for lda_cur in ldas_CV   ]
            means_list     = [lda_cur.means_  for lda_cur in ldas_CV   ]
            #rotations_[i]
            #scalings_[i]
            #means_[i]

            scalings_aver = sum(scalings_list) / len(scalings_list)
            rotations_aver = sum(rotations_list) / len(rotations_list)
            means_aver = sum(means_list) / len(means_list)


In [ ]:
lda.fit(Xconcat_good_cur, class_labels_good) 

In [ ]:
#lda.scalings_

In [ ]:
lda

In [ ]:
lda.coef_

In [ ]:
?? lda.decision_function

In [ ]:
lda_output_pg['merge_nothing'].keys()

In [ ]:
ldas_CV = lda_output_pg['merge_nothing']['trem_vs_quiet']['ldaobjs_CV']

In [ ]:
intercept_list     = [lda_cur.intercept_     for lda_cur in ldas_CV   ]
coef_list          = [lda_cur.coef_     for lda_cur in ldas_CV   ]

#rotations_list = [lda_cur.rotations_  for lda_cur in ldas_CV   ]
#means_list     = [lda_cur.means_  for lda_cur in ldas_CV   ]
#rotations_[i]
#scalings_[i]
#means_[i]

scalings_aver = sum(scalings_list) / len(scalings_list)
intercept_aver = sum(intercept_list) / len(intercept_list)
coef_aver = sum(coef_list) / len(coef_list)


In [ ]:
intercept_list

In [ ]:
from copy import deepcopy
lda2 = deepcopy(lda)

In [ ]:
lda2.coef_ = coef_aver
lda2.intercept_ = intercept_aver

In [ ]:
lda.coef_.shape

In [ ]:
lda2.intercept_

In [ ]:
#%debug
utsne.getLDApredPower(lda2,Xconcat_good_cur, class_labels_good, class_ind_to_check) 

In [ ]:
#/home/demitau/bigdisk/data/data_duss/S01_off_move_parcel_aal_grp9-0_all_PCA_nr2_7chs_nfeats1170_pcadim183_skip32_wsz256.npz


In [ ]:
#S01_off_hold_parcel_aal_grp9-0_all_PCA_nr2_7chs_nfeats1170_pcadim183_skip32_wsz256.npz

In [ ]:
del tSNE_result

In [ ]:
%run -i run_tSNE.py --prefix all --rawnames S01_off_hold,S01_off_move \
--sources_type parcel_aal --skip_calc 0

In [ ]:
perf_nocv_LDA, results_LDA, res_aver_LDA, ldas_CV = utsne.getPredPowersCV(lda_pri[0], Xconcat,class_labels, 
                      class_ind_to_check, printLog=False, n_splits=4, return_clf_obj=True)


In [ ]:
lda_pg_pri[0]['merge_nothing']['LDA']

In [ ]:
lda.scalings_.shape

In [ ]:
lda.coef_.shape

In [ ]:
gp.class_ids_def

In [ ]:
set(class_labels)

In [ ]:
set(class_labels_good)

In [ ]:
lda.scalings_.shape

In [ ]:
lda_pri[0].scalings_.shape

In [ ]:
lda_pri[1].scalings_.shape

In [ ]:
#lda_pri[2].scalings_.shape

In [ ]:
np.mean(np.vstack([[4, 5,  5], [5, 1 ,6]]), axis=0)

In [ ]:
if lda_pri[0].solver == 'svd':
    xbars = [  ]
    ldapts_ = np.dot(X - self.xbar_, self.scalings_)
elif lda_pri[0].solver == 'eigen':
    X_new = np.dot(X, self.scalings_)

In [ ]:
.scalings_
xbar_

In [ ]:
lda_pri[0].intercept_

In [ ]:
rawname_,sources_type,prefix

In [ ]:
filt_raw.ch_names)

In [ ]:
len(f.ch_names)

In [ ]:
filt_raw.info['bads']

In [ ]:
filt_raw.se

In [ ]:
#%debug
utils.collectChnamesBySide(f.info)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import mne
from mne.datasets.sleep_physionet.age import fetch_data
from mne.time_frequency import psd_welch

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer

In [ ]:
ALICE, BOB = 0, 1

[alice_files, bob_files] = fetch_data(subjects=[ALICE, BOB], recording=[1])

mapping = {'EOG horizontal': 'eog',
           'Resp oro-nasal': 'misc',
           'EMG submental': 'misc',
           'Temp rectal': 'misc',
           'Event marker': 'misc'}

raw_train = mne.io.read_raw_edf(alice_files[0])
annot_train = mne.read_annotations(alice_files[1])

raw_train.set_annotations(annot_train, emit_warning=False)
raw_train.set_channel_types(mapping)

# plot some data
raw_train.plot(duration=60, scalings='auto')

In [ ]:
annotation_desc_2_event_id = {'Sleep stage W': 1,
                              'Sleep stage 1': 2,
                              'Sleep stage 2': 3,
                              'Sleep stage 3': 4,
                              'Sleep stage 4': 4,
                              'Sleep stage R': 5}

events_train, _ = mne.events_from_annotations(
    raw_train, event_id=annotation_desc_2_event_id, chunk_duration=30.)

# create a new event_id that unifies stages 3 and 4
event_id = {'Sleep stage W': 1,
            'Sleep stage 1': 2,
            'Sleep stage 2': 3,
            'Sleep stage 3/4': 4,
            'Sleep stage R': 5}

# plot events
mne.viz.plot_events(events_train, event_id=event_id,
                    sfreq=raw_train.info['sfreq'])

# keep the color-code for further plotting
stage_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']